# 사용자 정의 모델 클래스 
- 부모 클래스 : nn.Module
- 필수 오버라이딩 
    - _ _init_ _() : 모델층 구성 즉, 설계
    - forward() : 순방향 학습 진행 코드 구현

In [47]:
import torch                                # 텐서 관련 모듈 
import torch.nn as nn                       # 인공신경망 관련 모듈 
import torch.nn.functional as F             # 인공신경망 관련 함수들 모듈 (손실함수, 활성화 함수 등등)
import torch.optim as optim                 # 최적화 관련 모듈 (가중치, 절편 빠르게 찾아주는 알고리즘 )
from torchinfo import summary               # 모델 구조 및 정보 관련 모듈 
from torchmetrics.regression import *       # 회귀 성능 지표 관련 모듈 
from torchmetrics.classification import *   # 분류 성능 지표 관련 모듈 

- [기본] 신경망 클래스 <hr>
    - 입력층 - 입력 : 피쳐수 고정 
    - 출력층 - 출력 : 타겟수 고정 
    - 은닉층 - 고정     

In [48]:
# 랜덤 고정 
torch.manual_seed(1)

# 텐서 저장 및 실행 위치 설정 
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [49]:
# 모델 설계 

# 데이터 셋 : 피쳐 4개, 타겟 1개, 회귀  
# 입 력 층 : 입력   4개,    출력  20개,     AF      ReLU(기울기 소실)
# 은 닉 층 : 입력  20개,    출력 100개,     AF      ReLU
# 출 력 층 : 입력  100개,   출력   1개,     AF      [회귀] X(회귀는 필요없음) , [분류] Sigmoid(딥러닝 - 이진분류) 또는 SoftMax(딥러닝 - 다중분류)

In [50]:
class MyModel(nn.Module):

    # 인스턴스/ 객체 생성 시 자동호출되는 메서드 (콜백함수 Callback Func)

    def __init__(self): # __x__ : 매직메서드 (파이썬 시스템에 의해 호출됨)
        # 부모클래스 생성 
        super().__init__()

        # 자식 클래스의 인스턴스 속성 설정 
        self.input_layer = nn.Linear(4,20)     # w 4 + b 1 => 1퍼셉트론, 5 * 20개의 변수 생성 
        self.hidden_layer = nn.Linear(20,100)  # w 20 + b 1 => 1퍼셉트론 , 21 * 100개의 변수 생성 
        self.output_layer = nn.Linear(100,1)   # w 100 + b 1 => 1퍼셉트론, 101 * 1 개의 변수 생성 

    # 순방향/전방향 학습 진행 시 자동 호출되는 메서드 (콜백함수 Callback Func : 시스템에서 호출되는 함수)
    # 전달 인자 : 학습용 데이터 셋

    def forward(self, x): # 데이터를 넣어야 함
        print('calling forward()')
        y = self.input_layer(x)     # 1개의 퍼셉트론 : y = x1w1 + x2w2 + x3w3 + x4w4 + b
        y = F.relu(y)               # 활성함수에 넣어서 결과값 반환 -> 0 <= y ----> 단점 : 죽은 relu 발생 -----> LeakyReLu 사용 

        y = self.hidden_layer(y)    # 1개의 퍼셉트론 : y = x1w1 + x2w2 ..... x20w20 + b
        y = F.relu(y)

        return self.output_layer(y) # 1개의 퍼셉트론 : y = x1w1 + x2w2 ..... x100w100 + b


위의 모델은 무조건 (4,1)가 되어야 하지만 아래 모델은 동적으로 변경 ㅇ 

In [51]:
# 입력 피쳐 수가 동적인 모델 


class MyModel2(nn.Module):

    # 인스턴스/ 객체 생성 시 자동호출되는 메서드 (콜백함수 Callback Func)

    def __init__(self, in_features): # __x__ : 매직메서드 (파이썬 시스템에 의해 호출됨)
        # 부모클래스 생성 
        super().__init__()

        # 자식 클래스의 인스턴스 속성 설정 
        self.input_layer = nn.Linear(in_features,20)      
        self.hidden_layer = nn.Linear(20,100)           
        self.output_layer = nn.Linear(100,1)            

    # 순방향/전방향 학습 진행 시 자동 호출되는 메서드 (콜백함수 Callback Func : 시스템에서 호출되는 함수)
    # 전달 인자 : 학습용 데이터 셋

    def forward(self, x): # 데이터를 넣어야 함
        print('calling forward()')
        y = self.input_layer(x)     
        y = F.relu(y)               

        y = self.hidden_layer(y)   
        y = F.relu(y)

        return self.output_layer(y) 

In [52]:
# 입력 피쳐 수, 은닉층 퍼셉트론 수가 동적인 모델 

class MyModel3(nn.Module):

    # 인스턴스/ 객체 생성 시 자동호출되는 메서드 (콜백함수 Callback Func)

    def __init__(self, in_features, in_out, h_out): # __x__ : 매직메서드 (파이썬 시스템에 의해 호출됨)
        # 부모클래스 생성 
        super().__init__()
        # 자식 클래스의 인스턴스 속성 설정 
        self.input_layer = nn.Linear(in_features,in_out)     
        self.hidden_layer = nn.Linear(in_out,h_out) 
        self.output_layer = nn.Linear(h_out,1)   

    # 순방향/전방향 학습 진행 시 자동 호출되는 메서드 (콜백함수 Callback Func : 시스템에서 호출되는 함수)
    # 전달 인자 : 학습용 데이터 셋

    def forward(self, x): # 데이터를 넣어야 함
        print('calling forward()')
        y = self.input_layer(x)    
        y = F.relu(y)               

        y = self.hidden_layer(y)    
        y = F.relu(y)

        return self.output_layer(y) 

In [53]:
# 은닉층의 개수가 동적인 모델 

class MyModel4(nn.Module):

    # 인스턴스/ 객체 생성 시 자동호출되는 메서드 (콜백함수 Callback Func)

    def __init__(self, in_features, in_out, h_cnt): # __x__ : 매직메서드 (파이썬 시스템에 의해 호출됨)
        # 부모클래스 생성 
        super().__init__()
        # 자식 클래스의 인스턴스 속성 설정 
        self.input_layer = nn.Linear(in_features,in_out)    

        n = 5
        if h_cnt != 0: 
            for i in range(1,h_cnt+1):
                
                self.hidden_layer = nn.Linear(in_out + (i-1)*n, in_out + i*n)
                print(f'(hidden_layer): Linear(in_features={in_out + (i-1)*n}, out_features={in_out + i*n}, bias=True)')
            
        if h_cnt == 0:
            self.output_layer = nn.Linear(in_out,1)  
        else:
            self.output_layer = nn.Linear(in_out + (h_cnt)*n,1)  
        

    # 순방향/전방향 학습 진행 시 자동 호출되는 메서드 (콜백함수 Callback Func : 시스템에서 호출되는 함수)
    # 전달 인자 : 학습용 데이터 셋

    def forward(self, x): # 데이터를 넣어야 함
        print('calling forward()')
        y = self.input_layer(x)     
        y = F.relu(y)               

        y = self.hidden_layer(y)    
        y = F.relu(y)

        return self.output_layer(y) 

ex_custom_model_02, 03 파일에서 배운 nn.ModuleList 적용 

In [54]:
# 은닉층의 개수가 동적인 모델 

class MyModel4_1(nn.Module):

    # 인스턴스/ 객체 생성 시 자동호출되는 메서드 (콜백함수 Callback Func)

    def __init__(self, in_features, in_out, h_cnt): # __x__ : 매직메서드 (파이썬 시스템에 의해 호출됨)
        # 부모클래스 생성 
        super().__init__()
        # 자식 클래스의 인스턴스 속성 설정 
        self.input_layer = nn.Linear(in_features,in_out)    

        self.hidden_layer = nn.ModuleList()
        n = 5
        if h_cnt != 0: 
            for i in range(1,h_cnt+1):
                
                self.hidden_layer.append(nn.Linear(in_out + (i-1)*n, in_out + i*n))
            
        if h_cnt == 0:
            self.output_layer = nn.Linear(in_out,1)  
        else:
            self.output_layer = nn.Linear(in_out + (h_cnt)*n,1)  
        

    # 순방향/전방향 학습 진행 시 자동 호출되는 메서드 (콜백함수 Callback Func : 시스템에서 호출되는 함수)
    # 전달 인자 : 학습용 데이터 셋

    def forward(self, x): # 데이터를 넣어야 함
        print('calling forward()')
        y = self.input_layer(x)     
        y = F.relu(y)               

        y = self.hidden_layer(y)    
        y = F.relu(y)

        return self.output_layer(y) 

In [55]:
# 모델 인스턴스 생성 
m1 = MyModel()

In [56]:
m1_3 = MyModel3(3,50,30) # 동적으로 생성 ㅇ 
m1_3

MyModel3(
  (input_layer): Linear(in_features=3, out_features=50, bias=True)
  (hidden_layer): Linear(in_features=50, out_features=30, bias=True)
  (output_layer): Linear(in_features=30, out_features=1, bias=True)
)

In [57]:
m2 = MyModel4(4,60,5)
m2

(hidden_layer): Linear(in_features=60, out_features=65, bias=True)
(hidden_layer): Linear(in_features=65, out_features=70, bias=True)
(hidden_layer): Linear(in_features=70, out_features=75, bias=True)
(hidden_layer): Linear(in_features=75, out_features=80, bias=True)
(hidden_layer): Linear(in_features=80, out_features=85, bias=True)


MyModel4(
  (input_layer): Linear(in_features=4, out_features=60, bias=True)
  (hidden_layer): Linear(in_features=80, out_features=85, bias=True)
  (output_layer): Linear(in_features=85, out_features=1, bias=True)
)

In [62]:
m2_m = MyModel4_1(4,60,5)
m2_m

MyModel4_1(
  (input_layer): Linear(in_features=4, out_features=60, bias=True)
  (hidden_layer): ModuleList(
    (0): Linear(in_features=60, out_features=65, bias=True)
    (1): Linear(in_features=65, out_features=70, bias=True)
    (2): Linear(in_features=70, out_features=75, bias=True)
    (3): Linear(in_features=75, out_features=80, bias=True)
    (4): Linear(in_features=80, out_features=85, bias=True)
  )
  (output_layer): Linear(in_features=85, out_features=1, bias=True)
)

In [59]:
m3 = MyModel4(4,40,0)
m3

MyModel4(
  (input_layer): Linear(in_features=4, out_features=40, bias=True)
  (output_layer): Linear(in_features=40, out_features=1, bias=True)
)

In [60]:
# 모델 파라미터 즉, w와 b 확인 

for m in m1.named_parameters(): print(m)

# class MyModel(nn.Module):
#   pass

# 만든게 없어서 출력 아무것도 안 됨 

('input_layer.weight', Parameter containing:
tensor([[ 0.2576, -0.2207, -0.0969,  0.2347],
        [-0.4707,  0.2999, -0.1029,  0.2544],
        [ 0.0695, -0.0612,  0.1387,  0.0247],
        [ 0.1826, -0.1949, -0.0365, -0.0450],
        [ 0.0725, -0.0020,  0.4371,  0.1556],
        [-0.1862, -0.3020, -0.0838, -0.2157],
        [-0.1602,  0.0239,  0.2981,  0.2718],
        [-0.4888,  0.3100,  0.1397,  0.4743],
        [ 0.3300, -0.4556, -0.4754, -0.2412],
        [ 0.4391, -0.0833,  0.2140, -0.2324],
        [ 0.4906, -0.2115,  0.3750,  0.0059],
        [-0.2634,  0.2570, -0.2654,  0.1471],
        [-0.1444, -0.0548, -0.4807, -0.2384],
        [ 0.2713, -0.1215,  0.4980,  0.4008],
        [-0.0234, -0.3337,  0.3045,  0.1552],
        [-0.3232,  0.3248,  0.3036,  0.4434],
        [-0.2803, -0.0823, -0.0097,  0.0730],
        [-0.3795, -0.3548,  0.2720, -0.1172],
        [ 0.2442,  0.0285,  0.1642,  0.1099],
        [ 0.1818,  0.2479, -0.4631,  0.2517]], requires_grad=True))
('input_layer

In [61]:
# 학습 진행 ==> 모델 인스턴스명(데이터)
# 임의의 데이터 생성 

data_ts = torch.FloatTensor([[1,3,5,7],[2,4,6,8]]) # Linear 모듈이 float을 지원하므로 float 사용 
# 설계한 shape에 맞게 개수 줘야 함 
# self.input_layer = nn.Linear(4,20) -> [[1,3,5],[2,4,6]] -> 오류 발생 
target_ts = torch.FloatTensor([[4],[5]])

# 학습 
pre_y = m1(data_ts)

print(pre_y)

calling forward()
tensor([[-0.1395],
        [-0.1858]], grad_fn=<AddmmBackward0>)


-> [1,3,5,7] 로 -0.1395, [2,4,6,8] 로 -0.1858예측